# Speedup Pandas

In [1]:
%load_ext autotime

time: 200 µs (started: 2022-10-12 13:47:35 -04:00)


In [2]:
# Alcohol available for consumption: Year ended December 2021 – CSV
# https://stats.govt.nz/large-datasets/csv-files-for-download/
# https://stats.govt.nz/assets/Uploads/Alcohol-available-for-consumption/Alcohol-available-for-consumption-Year-ended-December-2021/Download-data/alcohol-available-for-consumption-year-ended-december-2021-csv.csv

import pandas as pd

df = pd.read_csv("data/data.csv") # 11K

# multiply the data to have a bigger dataset
# df = pd.concat([df, df, df, df, df, df, df, df, df]) # 100K
# df = pd.concat([df, df, df, df, df, df, df, df, df, df]) # 1M
# df = pd.concat([df, df, df, df, df, df, df, df, df, df]) # 10M

df = df.reset_index(drop=True)
df["pk"] = df.index # primary key

print(f"Shape: {df.shape[0]:,} rows - {df.shape[1]:,} cols")
df.head()

Shape: 11,709 rows - 14 cols


,Series_reference,Period,Data_value,STATUS,UNITS,MAGNTUDE,Subject,Group,Series_title_1,Series_title_2,Series_title_3,Series_title_4,Series_title_5,pk
0,ALCA.S1,1935.12,51.1,FINAL,Litres,6,Alcohol Available for Consumption - ALC,(DISC) Volume & Volume Per Head,Beer,NaN,NaN,NaN,NaN,0
1,ALCA.S1,1936.12,58.7,FINAL,Litres,6,Alcohol Available for Consumption - ALC,(DISC) Volume & Volume Per Head,Beer,NaN,NaN,NaN,NaN,1
2,ALCA.S1,1937.12,68.7,FINAL,Litres,6,Alcohol Available for Consumption - ALC,(DISC) Volume & Volume Per Head,Beer,NaN,NaN,NaN,NaN,2
3,ALCA.S1,1938.12,78.1,FINAL,Litres,6,Alcohol Available for Consumption - ALC,(DISC) Volume & Volume Per Head,Beer,NaN,NaN,NaN,NaN,3
4,ALCA.S1,1939.12,79.2,FINAL,Litres,6,Alcohol Available for Consumption - ALC,(DISC) Volume & Volume Per Head,Beer,NaN,NaN,NaN,NaN,4


time: 204 ms (started: 2022-10-12 13:47:35 -04:00)


In [3]:
df.dtypes

Series_reference     object
Period              float64
Data_value          float64
STATUS               object
UNITS                object
MAGNTUDE              int64
Subject              object
Group                object
Series_title_1       object
Series_title_2      float64
Series_title_3      float64
Series_title_4      float64
Series_title_5      float64
pk                    int64
dtype: object

time: 1.17 ms (started: 2022-10-12 13:47:35 -04:00)


## pd.iterrows()

In [4]:
import calendar
import math
from tqdm.notebook import tqdm


df["heavy_math"] = pd.NA

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    period = str(row["Period"])
    pk = row["pk"]
    
    year, month = period.split(".")
    day = calendar.monthrange(int(year), int(month))[1]
    
    heavy_math = math.log((int(year) ** int(month)) * day)
    
    df.loc[df["pk"] == pk, 'heavy_math'] = heavy_math


  0%|          | 0/11709 [00:00<?, ?it/s]

time: 1.19 s (started: 2022-10-12 13:47:35 -04:00)


## pd.itertuples()

In [5]:
import calendar
import math
from tqdm.notebook import tqdm


df["heavy_math"] = pd.NA

for row in tqdm(df.itertuples(), total=df.shape[0]):
    period = str(row.Period)
    pk = row.pk
    
    year, month = period.split(".")
    day = calendar.monthrange(int(year), int(month))[1]
    
    heavy_math = math.log((int(year) ** int(month)) * day)
    
    df.loc[df["pk"] == pk, 'heavy_math'] = heavy_math


  0%|          | 0/11709 [00:00<?, ?it/s]

time: 945 ms (started: 2022-10-12 13:47:37 -04:00)


## pd.apply()

In [6]:
import calendar
import math

from tqdm.notebook import tqdm
tqdm.pandas()

df["heavy_math"] = pd.NA

def process(row):
    period = str(row["Period"])
    pk = row["pk"]
    
    year, month = period.split(".")
    day = calendar.monthrange(int(year), int(month))[1]
    
    heavy_math = math.log((int(year) ** int(month)) * day)
        
    return heavy_math

# using `df.progress_apply` instead of `df.apply` to show tqdm progress bar
df['heavy_math'] = df.progress_apply(lambda row : process(row), axis = 1)

  0%|          | 0/11709 [00:00<?, ?it/s]

time: 97.9 ms (started: 2022-10-12 13:47:38 -04:00)


## pandarallel

In [7]:
from pandarallel import pandarallel
import numpy as np
import math
import calendar

pandarallel.initialize(progress_bar=True)

df["heavy_math"] = pd.NA

def process_parallel(row):
    import calendar
    import math

    period = str(row["Period"])
    pk = row["pk"]
    
    year, month = period.split(".")
    day = calendar.monthrange(int(year), int(month))[1]
    
    try:
        heavy_math = math.log((int(year) ** int(month)) * day)
    except:
        heavy_math = np.inf
            
    return heavy_math

df['heavy_math'] = df.parallel_apply(process_parallel, axis=1)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


time: 138 ms (started: 2022-10-12 13:47:38 -04:00)


## Multiprocessing Pool

In [8]:
from src.multicore import parallelize_dataframe
import multiprocessing
import pandas as pd

df["heavy_math"] = pd.NA

# create as many processes as there are CPUs on your machine
num_processes = multiprocessing.cpu_count()

# apply our function to the dataframe
df = parallelize_dataframe(df, parallelize_dataframe, num_processes)

time: 583 ms (started: 2022-10-12 13:47:38 -04:00)


## Concurrent Future Pool

In [9]:
from src.multicore import parallelize_dataframe_future
import multiprocessing

df["heavy_math"] = pd.NA

# create as many processes as there are CPUs on your machine
num_processes = multiprocessing.cpu_count()

# apply our function to the dataframe
df = parallelize_dataframe_future(df, parallelize_dataframe_future, num_processes)

time: 515 ms (started: 2022-10-12 13:47:38 -04:00)


## Dask

In [10]:
import multiprocessing
import dask.dataframe as ddf

def proccess_dask(row):
    period = str(row["Period"])
    pk = row["pk"]
    
    year, month = period.split(".")
    day = calendar.monthrange(int(year), int(month))[1]
    
    heavy_math = math.log((int(year) ** int(month)) * day)
        
    return heavy_math


df["heavy_math"] = pd.NA

# create as many processes as there are CPUs on your machine
num_processes = multiprocessing.cpu_count()

# where the number of partitions is the number of cores you want to use
df_dask = ddf.from_pandas(df, npartitions=num_processes)

df_dask["heavy_math"] = df_dask.apply(lambda x: proccess_dask(x),
                                      axis=1,
                                      meta=pd.DataFrame).compute(scheduler='multiprocessing')

/opt/miniconda3/envs/pandas-speed-up/lib/python3.8/site-packages/dask/dataframe/core.py:6597: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


time: 813 ms (started: 2022-10-12 13:47:39 -04:00)


## Vectorization

Vectorization means to use `pandas` and `numpy` built-in functions. Among all those functions, the following functions are very useful to vectorize data processing in `pandas`:

- pd.cumsum()
- pd.shift()
- pd.ffill()
- pd.bfill()
- np.where()

And using `numpy` library for mathematical functions on `pandas` dataframe, such as:

- np.log()
- np.std()
- np.mean()
- etc.


In [11]:
import numpy as np
import pandas as pd

df = df.astype({'Period': 'str'})

df[['period_year', 'period_month']] = df['Period'].str.split('.', expand=True)

df = df.astype({
                'period_year': 'int',
                'period_month': 'int'
                })

df["period_day"] = pd.to_datetime(df['period_year'].astype(str) + '-' + df['period_month'].astype(str) + '-1',
                                  format='%Y-%m-%d').dt.days_in_month.astype(int)

df["heavy_math"] = np.log(df['period_year'].pow(df['period_month']) * df["period_day"])

time: 24.3 ms (started: 2022-10-12 13:47:40 -04:00)


/opt/miniconda3/envs/pandas-speed-up/lib/python3.8/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/miniconda3/envs/pandas-speed-up/lib/python3.8/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
